# GAIA PSF Simulation

First attempt to simulate close binary stars observation using the gaia telescope. The latter is imagined as a rectangular pupil, with no pixel binning or aberrations

#### Initialization

In [1]:
%matplotlib qt

QSocketNotifier: Can only be used with threads started with QThread


In [2]:
# %matplotlib qt
import logging
# logging.basicConfig(level=logging.WARNING, format=' - %(levelname)s - %(message)s')

In [3]:
import functools
import time

def timer(func):
    """Decorator to time the execution of a function."""

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        h = (end_time - start_time) // 3600
        m = (end_time - start_time) % 3600 // 60
        s = (end_time - start_time) % 60
        print(f"Execution time: {int(h):02d}:{int(m):02d}:{s:.2f} (h:m:s)")
        return result

    return wrapper

In [4]:
import poppy
import xupy as xp
from xupy import typing as xt
import astropy.units as u
from astropy import convolution as c
from astropy.io import fits
from astropy.table import QTable
from matplotlib import pyplot as plt
from opticalib import load_fits
from simulator import BinarySystem, CCD

Gpb = QTable.read("data/gaiaDR3passband.fits")
weights = Gpb.filled(0)

def rebinned(psf: fits.HDUList | xt.ArrayLike, rebin_factor: int) -> fits.HDUList:
    """
    Rebin PSF by a given factor, following Gaia's pixel scale (1:3 ratio).
    """
    if isinstance(psf, fits.HDUList):
        psf = psf[0].data
    px_ratio = (rebin_factor, 3*rebin_factor)
    return poppy.utils.rebin_array(psf, px_ratio)

/home/pietrof/miniforge3/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')



[XuPy] Device 0 available - GPU : `NVIDIA GeForce RTX 5080 Laptop GPU`
       Memory = 16230.98 MB | Compute Capability = 12.0
       Using CuPy 13.5.1 for acceleration.


```py
poppy.conf.use_multiprocessing = False
poppy.conf.n_processes = 20
poppy.conf.double_precision = False

M1 = poppy.RectangleAperture(name="Primary Mirror", height=1.45*u.m, width=0.5*u.m)
M2 = poppy.RectangleAperture(name="Beam Collimator / Secondary Mirror",width=0.35*u.m, height=0.2*u.m)
telescope = poppy.OpticalSystem(name="Gaia", oversample=1, verbose=True)

telescope.add_pupil(M1)
telescope.add_detector(name="SkyPLANE", pixelscale=0.001*(u.arcsec/u.pixel), fov_pixels=1125, oversample=4)

psf = telescope.calc_psf(progressbar=True, return_final=True, source={'wavelengths': Gpb['lambda'], 'weights': weights['G']})

poppy.utils.display_psf(psf[0], title="Gaia PSF", vmax=psf[0][0].data.max())
psf[0].writeto("data/simulations/PSFs/20250922_2_gaia_psf.fits", overwrite=True)

## Got The PSF: Now on to the convolution

#### Creating the **Binary Star System (BSS)** and the **CCD**

In [5]:
bs = BinarySystem(M1=7, M2=7, distance=200)
ccd = CCD(psf="data/simulations/PSFs/20250922_2_gaia_psf.fits")

bs._band_flux()
bs.create_raw_binary_cube(collecting_area=1.45*0.5*u.m**2,t_integration=4.42*u.s, shape=ccd.psf.shape)

MemoryError: Estimated memory (95.17 GB) exceeds 80% of available RAM (24.85 GB). Reduce shape or distance.

#### Loading the PSF and creating the CCD

In [ ]:
from astropy.visualization import ImageNormalize, MinMaxInterval, LogStretch
norm = ImageNormalize(vmin=xp.np.nanmin(ccd.psf), vmax=xp.np.nanmax(ccd.psf), stretch=LogStretch(), interval=MinMaxInterval())
plt.imshow(ccd.psf, origin='lower', cmap='viridis', norm=norm)
plt.colorbar(label='Intensity')
plt.title(f'Rebinned PSF : {ccd.psf.shape}')


**First try at observation**

In [ ]:
img = bs.map[0].copy()
psf = ccd.psf.copy()

cov_img_cpu = c.convolve_fft(img, psf, boundary='wrap', nan_treatment='fill', fill_value=0., allow_huge=True)

In [ ]:
plt.imshow(cov_img_cpu, origin='lower', cmap='viridis', aspect='auto')

In [ ]:
npsf = ccd.rebin_psf(cov_img_cpu, rebin_factor=1, axis_ratio=(3,1))

In [ ]:
plt.plot(npsf[2])

In [ ]:
plt.imshow(npsf[0], aspect='auto')

### GPU FFT-Convolve

In [ ]:
@timer
def gpu_covfft(img, psf, dtype = xp.float):
    img_g = xp.asarray(img, dtype=dtype)
    psf_g = xp.asarray(psf, dtype=dtype)
    img1 = xp.fft.fftn(img_g)
    psffft = xp.fft.fftn(xp.fft.ifftshift(psf_g))
    fftmult = img1 * psffft
    cov_img_gpu = xp.fft.ifftn(fftmult).real
    return cov_img_gpu

@timer
def cpu_covfft(img, psf):
    img_c = img.copy()
    psf_c = psf.copy()
    cov_img_cpu = c.convolve_fft(
        img_c, 
        psf_c, 
        normalize_kernel=True, 
        boundary='wrap', 
        nan_treatment='fill', 
        fill_value=0.0, 
        allow_huge=True
    )
    return cov_img_cpu